In [ ]:
import numpy as np
import math 

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Button, RadioButtonGroup, Slider, CheckboxButtonGroup
from bokeh.plotting import figure, show, output_file

In [ ]:
# Scatterplot: x from uniform(0,1); y from normal(0,1)
# Add 1 particle on click
# Send 100 particles at once
# Reset the screen
# Continuous stream of particles + jde vypnout
# User-defined distribution function + histogram

max_x = 10  # Délka stínítka
var_x = 1  #  Rozptyl normálního rozdělení
slitdist = 0.1 # Vzdálenost štěrbin 
slitwidth = 0.1 # Šířka štěrbin
source = ColumnDataSource(data=dict(x=[], y=[]), tags=[dict(max_x=max_x, var_x=var_x, slitdist=slitdist, slitwidth=slitwidth)])


# Screen with points
p = figure(
    title="Screen of the Young double slit experiment",
    tools="pan,wheel_zoom,box_zoom,reset",         # (de)activate tools    
    sizing_mode="stretch_width", 
#    max_width=1000, 
    width=500,
    height=500,
    x_axis_label="x",
    y_axis_label="y",
    x_range=(0,max_x),
    y_range=(0, 1),
)
p.circle('x', 'y', source=source, size=10, fill_color="red")

# Probability denstity function above the screen + histogram
pd = figure(title="Probability denstity function of Young double slit experiment",
            sizing_mode="stretch_width", 
            width=p.width, height=250, x_range=p.x_range)

# Theoretical probability density function -- young double slit
amp = 1  # ------------> jak předávat hodnoty těchto konstant do js funkce youngPDF(x)?
freq = 1
x = np.linspace(0, max_x, 100)
young_pdf = amp*np.cos(freq*x)**2
source_pd = ColumnDataSource(data=dict(x=x, y=young_pdf), tags=[dict(amp=amp, freq=freq)])  
pd.line('x', 'y', source=source_pd, line_width=2, line_color="navy", legend_label="Probability Density Function")

# Histogram -- young double slit
Nbins = 10 # number of bins
binwidth = max_x/Nbins
hist, bins = np.histogram([], range=(0, max_x), bins=Nbins) 
# hist = rel. četnosti, bins = array dělicích bodů binů, counts = abs četnosti
source_hist = ColumnDataSource(data=dict(hist=[0 for _ in range(Nbins)], 
                                         bins=[binwidth/2 + i*binwidth for i in range(Nbins)], # dělení vystředované
                                         counts=[0 for _ in range(Nbins)]), # array abs četností
                               tags=[dict(binwidth=binwidth)])  
pd.vbar(top="hist", x="bins", width=binwidth, source=source_hist, alpha=0.6, color="skyblue")

# Buttons ----------------------------------------------------------------------
# Add 1 particle
button = Button(label="Add 1 particle", button_type="success")

# Send 100 particles at once
button100 = Button(label="Send 100 particles", button_type="success")

# Reset the screen
resetbutton = Button(label="Reset", button_type="success")

# Continuous stream of particles
streamtoggle = Button(label="Send continuous stream", button_type="success")

# Change distribution
LABELS = ["Uniform", "Normal", "Young"]
button_change_dist = RadioButtonGroup(labels=LABELS, active=0)

# Block/open slits
LABELS = ["Slit 1", "Slit 2"]

button_choose_slit = CheckboxButtonGroup(labels=LABELS, active=[0, 1])

#Slider
slider_amp = Slider(start=0, end=5, value=1, step=0.1, title="Amplitude")
slider_freq = Slider(start=0, end=5, value=1, step=0.1, title="Frequency")
slider_slitdist = Slider(start=0, end=2, value=0.1, step=0.1, title="Slit distance")
slider_slitwidth = Slider(start=0, end=2, value=0.1, step=0.1, title="Slit width")

callback_young_Teorpdf = CustomJS(args=dict(source_pd=source_pd, slider_amp=slider_amp, slider_freq=slider_freq, 
                                            slider_slitdist=slider_slitdist, slider_slitwidth=slider_slitwidth),
                    code="""
    const amp = slider_amp.value
    const freq = slider_freq.value
    const slitdist = slider_slitdist.value
    const slitwidth = slider_slitwidth.value

    const x = source_pd.data.x
    const y = Array.from(x, (x) => slitdist*amp*Math.cos(slitwidth*freq*x)**2 )
    console.log(y);
    source_pd.data = { x, y }
    //source_pd.change.emit()
    
    // push slider values to source_pd to be available for prob. distribution 
    //-----> ? nešlo by vefunkcích psát přímo slider_amp.value ?
    source_pd.tags[0].amp.push(slider_amp.value)
    source_pd.tags[0].freq.push(slider_freq.value)
    source_pd.change.emit()
    
    source.tags[0].slitdist.push(slider_slitdist.value)
    source.tags[0].slitwidth.push(slider_slitwidth.value)
    source.change.emit()
""")

slider_amp.js_on_change('value', callback_young_Teorpdf)
slider_freq.js_on_change('value', callback_young_Teorpdf)
slider_slitdist.js_on_change('value', callback_young_Teorpdf)
slider_slitwidth.js_on_change('value', callback_young_Teorpdf)
# -----------------------------------------------------------------------------
# JS functions definition

# # Def function of normal distribution
# js_normalPDF = """
#     function randn() { // Box Muller Transform
#       let u = 0, v = 0;
      
#       while(u === 0) u = Math.random(); //Converting [0,1) to (0,1)
#       while(v === 0) v = Math.random();
      
#       let num = Math.sqrt( -2.0 * Math.log( u ) ) * Math.cos( 2 * Math.PI * v );
#       num = num*source.tags[0]['var_x'] + source.tags[0]['max_x']/2;           // Shift mean  0 -> max_x/2; var = 1 -> var_x

#       if (num < 0) {return randn()}  // resample between 0 and max_x (abandon values out of screen)
#       else if (source.tags[0]['max_x'] < num) {return randn();}
#       else {return num;}
#     }
#  """

# Def function of normal distribution with parameters mean, var read from sliders
js_normalPDF = """
    function randn(mean, variance) { // Box Muller Transform
      let u = 0, v = 0;
      
      while(u === 0) u = Math.random(); //Converting [0,1) to (0,1)
      while(v === 0) v = Math.random();
      
      let num = Math.sqrt( -2.0 * Math.log( u ) ) * Math.cos( 2 * Math.PI * v );
      //num = num*source.tags[0]['var_x'] + (source.tags[0]['max_x']/2 - source.tags[0]['slitdist']); // Shift mean  0 -> max_x/2; var = 1 -> var_x
      num = num * variance + mean
      
      if (num < 0) {return randn()}  // resample between 0 and max_x (abandon values out of screen)
      else if (source.tags[0]['max_x'] < num) {return randn();}
      else {return num;}
    }
 """

# Def function of normal distribution (cases: both slits open, slit 1 open, slit 2 open)
# # Nehází chybu v konzoli, ale nefunguje. --------> Zprovoznit, hodnotu mean brát jako parametr slideru
# js_normalPDF = """
#     function randn() { // Box Muller Transform
#       let u = 0, v = 0;
      
#       while(u === 0) u = Math.random(); //Converting [0,1) to (0,1)
#       while(v === 0) v = Math.random();
      
#       let num = Math.sqrt( -2.0 * Math.log( u ) ) * Math.cos( 2 * Math.PI * v );
#       num = num*source.tags[0]['var_x'] + source.tags[0]['max_x']/2;           // Shift mean  0 -> max_x/2; var = 1 -> var_x

#       if (slit.active == [0,1]) { // both slits, Normal(max_x, var_x)
#           num = num*source.tags[0]['var_x'] + source.tags[0]['max_x']/2; 
          
#           if (num < 0) {return randn()}  // resample between 0 and max_x (abandon values out of screen)
#           else if (source.tags[0]['max_x'] < num) {return randn();}
#           else {return num;}
#           } 
#       else if (slit.active == [0]) { // only slit 1 active, Normal(max_x/4, var_x)
#           num = num*source.tags[0]['var_x'] + source.tags[0]['max_x']/4; 
          
#           if (num < 0) {return randn()}  // resample between 0 and max_x (abandon values out of screen)
#           else if (source.tags[0]['max_x'] < num) {return randn();}
#           else {return num;}
#           } 
#       else if (slit.active == [1]) { // only slit 2 active, Normal(3*max_x/4, var_x)
#           num = num*source.tags[0]['var_x'] + source.tags[0]['max_x']*3/4; 
#       }
#       else {}
#     }
#  """

# Def function of Young double slit interference distribution
js_youngPDF = """
    function youngPDF(x) { // PDF of Young interferention
        const amp = 1;
        const freq = 1;
        return amp*Math.cos(freq*x)**2
        
// nefunguje předávání hodnot ze slideru
//      return source_pd.tags[0].amp*Math.cos(source_pd.tags[0].freq*x)**2
    }
    
    let minI = 0;
    let N = 100;
    
    function youngCDF(maxI) { // CDF of Young interferention
      let integral = 0;
      let delta = (maxI - minI)/N
      for (let i = 0; i < N; i++) {
          integral += youngPDF(delta/2 + i*delta)*delta;
      } 
      return integral
    }
    
    const sample = [];    // array of divistion points of the x-axis (up to max_x, step 0.01)
    for (let i = 0; i < source.tags[0].max_x; i += 0.01) {
      sample.push(i);
    }
    const sample_y = sample.map(youngCDF);  // youngCDF function values = the division points of the y-axis
    
    // Normalization of PDF to the length of shade (max_x)
    const h = sample_y[sample_y.length-1] - sample_y[0];

    let index;
    function youngInverseCDF(t) { // inverse CDF of Young interferention
        for (let i = 0; i < sample_y.length; i++) {
            if (sample_y[i] > t) {
            index = i;
            break;
            }
        }
        const x2 = sample[index];
        const x1 = sample[index - 1];
        const y1 = sample_y[index - 1]
        const c = (x2 - x1)/(sample_y[index] + y1);
        return c*t - c*y1 + x1;
    }
"""

# Add one particle 
js_send = """
    function send(u, renormalize=true) {
        source.data.x.push(u)
        source.data.y.push(Math.random())
        source.change.emit()  // update the data source with local changes
        
        // add to histogram
        let j = Math.floor((u)/s_hist.tags[0].binwidth);  // index of bin where a particle with coord. new_x is added
        s_hist.data.counts[j]++;
        if (renormalize) {normalizehist()}; 
    }
    
    // normalize histogram
    function normalizehist() {
       // normalization; number of particles = source.data.x.length
       let total = source.data.x.length; // total number of particles
        for (let i = 0; i < s_hist.data.hist.length; i++) {
            s_hist.data.hist[i] = s_hist.data.counts[i]/total;
        }
        s_hist.change.emit();
        console.log("normalised");
    }
"""

# Generator of a random number from a given distribution (uniform / normal(max_x/2, var_x) / young)
# js_generator = """
#     function generator() {
#         if (selected.active == 0) {return source.tags[0].max_x*Math.random()}    // uniform distribution (0,1)
#         else if (selected.active == 1) {
#             return randn(source.tags[0].max_x/2 - 10*source.tags[0].slitdist, source.tags[0].var_x)
#         }     // normal distribution (0,1)
#         else {return youngInverseCDF(h*Math.random())} // young interference distribution
#     }
# """

# Generator of a random number from a given distribution (normal(mean, variance) / young)
# ---------> dostat sem parametry mean, variance ze sliderů: mean = max_x/2 - slitdist
js_generator = """
    
    function generator() {
        if (slit.active == [0,1]) { // sum of two normal dist N(max_x/4,1) + N(max_x*3/4,1)
            if (Math.random() < 0.5) { // randmoly switching between slit 1 and slit 2
                return randn(source.tags[0].max_x/2 - 10*source.tags[0].slitdist, source.tags[0].var_x)
            }
            else {
                return randn(source.tags[0].max_x/2 + 10*source.tags[0].slitdist, source.tags[0].var_x)
            }
        }  
        else if (slit.active == [0]) { // normal distribution (max_x/4,1)
            return randn(source.tags[0].max_x/2 - 10*source.tags[0].slitdist, source.tags[0].var_x)
        }
        else if (slit.active == [1]) { // normal distribution (max_x*3/4,1)
            return randn(source.tags[0].max_x/2 + 10*source.tags[0].slitdist, source.tags[0].var_x)
        }
        else {return youngInverseCDF(h*Math.random())} // young interference distribution
    }    
"""

#-------------------------------------------------------------------------
# JS CALLBACKS
js_functions = js_normalPDF + js_youngPDF + js_send + js_generator

callback = CustomJS(args=dict(source=source, s_hist=source_hist, selected=button_change_dist, slit=button_choose_slit), 
                    code=js_functions+"""
    send(generator())
    console.log(slit.active);
    """)

callback100 = CustomJS(args=dict(source=source, s_hist=source_hist, selected=button_change_dist, slit=button_choose_slit), 
                       code=js_functions+"""   
    for (let i = 0; i < 100; i++) {
        send(generator(), false)
    }
    normalizehist()
    console.log(selected.active)
    """)

callback_reset = CustomJS(args=dict(source=source, s_hist=source_hist, selected=button_change_dist, slit=button_choose_slit), 
                          code="""  
    // reset data in source
    source.data.x = []
    source.data.y = []
    source.change.emit()
    // reset histogram
    for (let i = 0; i < s_hist.data.hist.length; i++) {
            s_hist.data.hist[i] = 0;
            s_hist.data.counts[i] = 0;
            }
    s_hist.change.emit()
    """)

callback_stream = CustomJS(args=dict(source=source, streamtoggle=streamtoggle, s_hist=source_hist, selected=button_change_dist, 
                                     slit=button_choose_slit), 
                           code=js_functions+"""
    function sendparticle() {
        send(generator())
    }
      
    if (streamtoggle.button_type == "success") {
        const interval = setInterval(sendparticle, 1000);
        source.interval = interval;
        streamtoggle.button_type = "danger";
        streamtoggle.label = "Stop stream";
        
    } else {
        clearInterval(source.interval);
        streamtoggle.button_type = "success";
        streamtoggle.label = "Send continuous stream";
    }
""")

# Zapínání a vypínání štěrbin mění barvu a nápis tlačítek
# callback_choose_slit = CustomJS(args=dict(source=source, button_choose_slit=button_choose_slit, slit=button_choose_slit), 
#                            code=js_functions+      
#     if (button_choose_slit.button_type == "success") {
#         button_choose_slit.button_type = "danger";
#         button_choose_slit.label = "Slit 1 blocked";
        
#     } else {
#         button_choose_slit.button_type = "success";
#         button_choose_slit.label = "Slit 1 open";
#     }

# """)

# -------------------------------------------------------------------------------
# Callbacks to objects

button.js_on_event('button_click', callback)
button100.js_on_event('button_click', callback100)
resetbutton.js_on_event('button_click', callback_reset)
streamtoggle.js_on_event('button_click', callback_stream)
# button_change_dist.js_on_click('button_click', callback_)
# button_choose_slit.js_on_click('button_click', callback_choose_slit)

#buttons_layout = column(button, button100, resetbutton)
#show(row(p, buttons_layout))
show(row(column(pd, p), 
         column(button, button100, resetbutton, streamtoggle, 
                button_change_dist, button_choose_slit, 
                slider_amp, slider_freq, 
                slider_slitdist, slider_slitwidth)))
# slider_amp, slider_freq